## keywords for zero shot

- I, think -----> conversational

# `1) Setup`

### `1.1) Speech Recognition - setup`

In [1]:
import speech_recognition as sr

In [2]:
def takeCommand():
    #It takes microphone input from the user and returns string output    
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        r.pause_threshold = 1
        audio = r.listen(source)
        try:
            print("Recognizing...")    
            query = r.recognize_google(audio, language='en-in') #Using google for voice recognition.
            print(f"User said: {query}\n")  #User query will be printed.    
        except Exception as e:
            # print(e)  use only if you want to print the error!
            print("Say that again please...")   #Say that again will be printed in case of improper voice 
            return "None" #None string will be returned
        return query

### `1.2) NLU - setup`

In [3]:
from src.NLU.intention import IntentRecognizer
import pickle

In [4]:
from transformers import pipeline, Conversation

from parrot import Parrot
import torch
import warnings
warnings.filterwarnings("ignore")

In [5]:
with open('Saved/intention_models.pickle', 'rb') as handle:
    intention_models_trained_b = pickle.load(handle)
with open('Saved/tfidfV.pickle', 'rb') as handle:
    tfidfv_b = pickle.load(handle)
with open('Saved/les.pickle', 'rb') as handle:
    les_b = pickle.load(handle)

In [6]:
ir_b = IntentRecognizer()

In [7]:
# use models trained
ir_b.intent_models = intention_models_trained_b

In [8]:
transcript_eg = "turn off music"
ir_b.get_intents(transcript_eg, les_b, tfidfv_b)

{'action': 'deactivate', 'object': 'music', 'location': 'none'}

In [29]:
# 2 - zero shot model
# zs_model = pipeline("zero-shot-classification", model="joeddav/xlm-roberta-large-xnli")
zs_model = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

def get_discourse(query_in, labels_in):
    zsc_result = zs_model(query_in, labels_in)
    zsc_labels = zsc_result["labels"]  
    zsc_scores = zsc_result["scores"]
    discourse_prediction = zsc_labels[zsc_scores.index(max(zsc_scores))]
    return discourse_prediction

In [10]:
# 3 - conversational
converser = pipeline("conversational",
                      model="facebook/blenderbot-400M-distill")
# query_test = "I like going to shops"
# conversation = Conversation(query_test)
# response = converser(conversation).generated_responses[-1]

In [11]:
# 4- paraphrasing
# paraphraser = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5", use_gpu=False)

# phrases = ["Can you recommed some upscale restaurants in Newyork?",
#            "What are the famous places we should not miss in Russia?"
# ]

# for phrase in phrases:
#     print("-"*100)
#     print("Input_phrase: ", phrase)
#     print("-"*100)
#     para_phrases = paraphraser.augment(input_phrase=phrase)
#     for para_phrase in para_phrases:
#         print(para_phrase)

In [12]:
# 5 - storytelling
# from transformers import pipeline

# story_gen = pipeline("text-generation", "pranavpsv/gpt2-genre-story-generator")
# print(story_gen("<BOS> <superhero> Batman"))

### `1.3) Dialog Management and State Tracking - setup`

In [13]:
from src.dialog_manager.manage import manage_dialog

In [14]:
initial_states = {"music": {"none": "off"},
          "lights": {"none": "on", "bedroom": "on", "washroom":"on", "kitchen": "on"},
          "volume": {"none":"3"},
          "heat": {"none":"warm", "bedroom":"warm", "washroom":"warm", "kitchen": "warm"},
          "lamp": {"none": "off"},
          "newspaper": {"none": "not brought"},
          "juice": {"none": "not brought"},
          "socks": {"none": "not brought"},
          "chinese": {"none": "off"},
          "korean": {"none": "off"},
          "english": {"none": "on"},
          "german": {"none": "off"},
          "shoes": {"none": "not brought"}}

In [15]:
states = initial_states
action_prompt, object_prompt, volume_prompt = 'increase', 'volume', 'none'
current_intent_test = {'action': 'increase', 'object': 'volume', 'location': 'none'}
states_main, prompt_main, prev_states_main = manage_dialog(current_intent_test, states)

### `1.4) Text-to-Speech - setup`

In [16]:
import pyttsx3

In [17]:
# setting up the voice
engine = pyttsx3.init('sapi5')
voices = engine.getProperty('voices') #gets you the details of the current voice

engine.setProperty('voice', voices[1].id)  # 0-male voice , 1-female voice
engine.setProperty('rate', 200) 

In [18]:
def speak(audio):   
    engine.say(audio)    
    engine.runAndWait()  

In [19]:
speak("hello")

### `1.5) NLG - setup`

In [20]:
def greet_time():
    hour = int(datetime.datetime.now().hour)
    if hour>=0 and hour<12:
        speak("Good Morning!")    
    
    elif hour>=12 and hour<18:
        speak("Good Afternoon!")       
    
    else:
        speak("Good Evening!")   

### `1.6) Other Libraries - setup`

In [21]:
import webbrowser
import pyttsx3
import datetime
import wikipedia
import os

---

# `2) Running`

In [23]:
discourse_labels = ["conversational", "command"]


In [31]:
greet_time()
prev_response = ""

while True:
    query = takeCommand().lower() #Converting user query into lower case
    print("QUERY: {}".format(query))
    discourse_prediction = get_discourse(query_in=query, labels_in=discourse_labels)
    print(discourse_prediction)
    
    
    if query == "none" or len(query) == 0:
        continue
    
    if discourse_prediction == "command":
        # Logic for executing tasks based on query
        if 'wikipedia' in query:  #if wikipedia found in the query then this block will be executed
            speak('Searching Wikipedia...')
            query = query.replace("wikipedia", "")
            if query == "":
                query = "nothing"
            else:
                query = "what is life?"
            results = wikipedia.summary(query, sentences=5)
            print(results)
            response = "According to Wikipedia, {}".format(results)
            speak(response)

        elif 'open youtube' in query:
            webbrowser.open("youtube.com")
            response = "youtube opened"
            speak(response)

        elif 'open google' in query:
            webbrowser.open("google.com")
            response = "google opened"
            speak(response)

        elif 'play music' in query:
            music_dir = 'music_dir_of_the_user'
            songs = os.listdir(music_dir)
            print(songs)    
            os.startfile(os.path.join(music_dir, songs[0]))
            response = "playing music"

        elif 'the time' in query:
            strTime = datetime.datetime.now().strftime("%H:%M:%S")
            response = f"Sir, the time is {strTime}"
            speak(response)

        elif ('open stackoverflow' in query) or ('open stack overflow' in query):                           
            webbrowser.open('stackoverflow.com')
            response = "stackoverflow opened"

        elif ('open free code camp' in query) or ('open freecodecamp' in query):            
            webbrowser.open('freecodecamp.org')
            response = "freecodecamp opened"
            
        else:
            response = "sorry I didn't understand your command, can you repeat that please?"
            speak(response)
            
    
    elif discourse_prediction == "conversational":
        conversation = Conversation(query)
        response = converser(conversation).generated_responses[-1]
        speak(response)
    
    prev_response = response

Listening...
Recognizing...
result2:
{   'alternative': [{'confidence': 0.97219545, 'transcript': 'open YouTube'}],
    'final': True}
User said: open YouTube

QUERY: open youtube
command
Listening...
Recognizing...
result2:
[]
Say that again please...
QUERY: none
command
Listening...
Recognizing...
result2:
[]
Say that again please...
QUERY: none
command
Listening...


KeyboardInterrupt: 